<a href="https://colab.research.google.com/github/tohkunhao/DL-Library/blob/main/modular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import GPUtil
import numpy as np
import cupy as cp

In [ ]:
#def CheckGPU():
#  try:
#    GPUtil.getAvailable()
#    status="available"
#  except:
#    status="not available"
#  
#  return status


In [2]:
class nnModule():
  '''
  class containing defaults of every NN module.
  Default behaviour is that the module behaves the same when training and testing.
  ニューラルネットワークのベースクラス。
  デフォールトは学習とテスト時の処理が一緒。
  '''
  delta = 1e-07
  def __init__(self):
    self.params = {}
    self.grads = {}
    self.xp = None

  def eval(self): #to be present in every NN module
    pass
  
  def train(self): #to be present in every NN module
    pass

In [27]:
class Linear(nnModule):
  '''
  takes in the arguments (in_channels, out_channels, init_type, bias)
  in_channels is the number of input features
  out_channels is the number of perceptrons
  init_type is the type of weight initialisations. Default is He Kaiming's for use with ReLU
    other options include Xavier for tanh.
  bias determines if bias is used. Default is set to true.
  updateflg will determine if weights will be updated by the optimizer. Default is set to true.
  '''
  def __init__(self, in_channels, out_channels, init_type='He',bias=True,updateflg=True):
    super(Linear,self).__init__()
    self.in_channels = in_channels
    self.out_channels = out_channels
    self.init_type = init_type
    self.bias = bias
    self.updateflg = updateflg
  
  def forward(self,x):
    self.xp = cp.get_array_module(x)
    self.x = x #store x for use in backprop

    if self.init_type == 'He':
      sd = self.xp.sqrt(2/self.in_channels)
    elif self.init_type == 'Xavier':
      sd = self.xp.sqrt(1/self.in_channels)

    #initialise weights and biases
    if 'w' not in self.params.keys():
      self.params['w'] = self.xp.random.rand(self.in_channels,self.out_channels)*sd
    if self.bias and 'b' not in self.params.keys():
      self.params['b'] = self.xp.zeros(self.out_channels)#initialize at 0

    if self.bias:
      out = self.xp.dot(x,self.params['w'])+self.params['b']
    else:
      out = self.xp.dot(x,self.params['w'])
    
    return out

  def backward(self,dout):
    if self.bias:
      self.grads['b'] = self.xp.sum(dout,axis=0)
    
    self.grads['w'] = self.xp.dot(self.x.reshape((self.in_channels,-1)),dout.reshape((-1,dout.shape[-1]))) #to prevent errors in batch size 1

    return self.xp.dot(dout,self.xp.transpose(self.params['w']))


In [ ]:
class ReLU(nnModule):
  '''
  Activation function ReLU
  f(x)=x x>0, 0 x<0
  '''
  def __init__(self):
    super(ReLU,self).__init__()
    self.mask = None
    self.updateflg = False #no trainable parameters
  
  def forward(self,x):
    self.mask = x>0 #store for backprop
    self.xp = cp.get_array_module(x)
    return self.xp.maximum(0,x)
  
  def backward(self,dout):
    out = self.xp.zeros_like(dout)
    out[self.mask] = dout[self.mask]
    return out

In [ ]:
class Dropout(nnModule):
  '''
  Performs dropout on the outputs of the linear layer according to the
  probability of p as defined by the user
  '''
  def __init__(self,p=0.5):
    super(Dropout,self).__init__()
    self.p = p
    self.trainflg = True
    self.mask = None
    self.updateflg = False
  
  def forward(self,x):
    if self.trainflg:
      self.xp = cp.get_array_module(x)

      rng = self.xp.random.rand(*x.shape)
      out = self.xp.zeros_like(x)
      self.mask = rng > self.p
      out[self.mask] = x[self.mask]
    else:
      out = x * (1 - self.p) #scale the output at test time

    return out

  def backward(self,dout):
    out = self.xp.zeros_like(dout)
    out[self.mask] = dout[self.mask]
    return out
  
  def eval(self):
    self.trainflg = False #no dropout during test time
  
  def train(self):
    self.trainflg = True

In [ ]:
class BatchNorm(nnModule):
  '''
  Performs batchnorm on NxD tensor from 2015 batchnorm paper by Ioffe & Szegedy.
  NOT TO BE USED FOR CONVOLUTION. See BatchNormConv for convolutions.
  N being mini batch size, D being number of features
  '''
  def __init__(self):
    super(BatchNorm,self).__init__()
    self.trainflg = True
    self.updateflg = True
    self.mean = [] 
    self.var = []
    self.m = None
    self.x_hat = None
    self.mbmean = None
    self.mbvar = None
    self.x = None
  
  def forward(self,x):
    self.xp = cp.get_array_module(x)
    self.x = x #store for backprop

    #initialise the trainable parameters
    if not self.params.has_key('gamma'):
      self.params['gamma'] = self.xp.ones(x.shape[-1]) #as many as input features
    if not self.params.has_key('beta'):
      self.params['beta'] = self.xp.zeros(x.shape[-1]) #as many as input features

    if not self.m:
      self.m = x.shape[0] #set minibatch size if self.m is None

    if self.trainflg:
      self.mbmean = self.xp.mean(x, axis = 0)
      self.mbvar = self.xp.mean((x-self.mbmean)**2,axis = 0)
      self.mean.append(self.mbmean) #used to calculate moving average
      self.var.append(self.mbvar) #used to calculate moving average
      self.x_hat = (x - self.mbmean)/self.xp.sqrt(self.mbvar + delta)
    else:
      Ex = self.xp.mean(self.mean,axis = 0)#moving average
      Varx = (self.m/(self.m-1)) * self.xp.mean(self.var, axis = 0)#moving average
      self.x_hat = (x - Ex)/self.xp.sqrt(Varx + delta)
    
    return self.params['gamma'] * self.x_hat + self.params['beta']
    
  def backward(self,dout):
    #store trainable gradients
    self.grads['beta'] = self.xp.sum(dout, axis = 0) #(D,) dimension
    self.grads['gamma'] = self.xp.sum(dout*self.x_hat, axis = 0) #(D,) dimension

    #Not trainable gradients. No need to store
    dx_hat = self.params['gamma']*dout #(N,D) dimension
    dmbvar = self.xp.sum(dx_hat*(self.x-self.mbmean)*-0.5*self.xp.sqrt(self.mbvar+delta)**-3,axis = 0) #(D,) dimension
    dmbmean = self.xp.sum(dx_hat*-1/self.xp.sqrt(self.mbvar+delta),axis = 0)
    dmbmean += self.mbvar*self.xp.sum(-2*(self.x-self.mbmean),axis = 0)/self.m #(D,) dimension
    return dx_hat/self.xp.sqrt(self.mbvar+delta) + dmbvar*2*(self.x-self.mbmean)/self.m + dmbmean/self.m #(N,D) dimension
  
  def eval(self):
    self.trainflg = False
  
  def train(self):
    self.trainflg = True

In [47]:
class Sequence(nnModule):
  '''
  class for executing nnModules in a sequential manner (left to right).
  モジュールを順番に実行するクラス。
  '''
  def __init__(self,*layers):
    super(Sequence,self).__init__()
    self.modules = list(layers)
    self.x = None

  def forward(self,x):
    self.x = x
    for module in self.modules:
      x = module.forward(x)
    return x
  
  def backward(self,dout):
    for module in self.modules[::-1]:
      dout = module.backward(dout)
    return dout

  #need to add a way to update gradients of models contained in the sequence
